<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/chatBOT_memor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/hwchase17/chat-your-data
# https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook
# https://langchain.readthedocs.io/en/latest/modules/indexes/vectorstore_examples/faiss.html

!pip install -q langchain==0.0.100 openai unstructured faiss-cpu gradio tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.1/343.1 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 KB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.8 MB/s eta 0:00:

In [3]:
from getpass import getpass
import pickle

import tiktoken

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Load Data
loader = UnstructuredFileLoader("/content/FAQ_data.txt")
raw_documents = loader.load()

# Split text

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

# text_splitter = CharacterTextSplitter(chunk_size=2000)

documents = text_splitter.split_documents(raw_documents)

# Load Data to vectorstore

OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(documents, embeddings)

# Save vectorstore
with open("/content/vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

OpenAI API key: ··········


In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import ChatVectorDBChain

_template = """Dado um diálogo e uma pergunta, reformule a pergunta para ser uma pergunta independente.
Você pode supor que a pergunta seja sobre conteúdo do banco itaú.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


template = """Você é um chatbot que responde apenas perguntas sobre o banco Itaú.
Se você não souber a resposta, apenas diga "Hmm, não tenho certeza.". Não invente uma resposta.
Se a pergunta não é sobre o banco Itaú, informe educadamente que você só responde perguntas sobre o banco Itaú.
Question: {question}
=========
{context}
=========
Resposta em Markdown:"""
QA_PROMPT = PromptTemplate(template=template, input_variables=["question", "context"])

def get_chain(vectorstore):
    llm = OpenAI(model_name='gpt-3.5-turbo', openai_api_key=OPENAI_API_KEY, temperature=0)
    qa_chain = ChatVectorDBChain.from_llm(
        llm,
        vectorstore,
        qa_prompt=QA_PROMPT,
        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    )
    return qa_chain



if __name__ == "__main__":
    with open("/content/vectorstore.pkl", "rb") as f:
        vectorstore = pickle.load(f)
    qa_chain = get_chain(vectorstore)
    chat_history = []
    print("Faça uma pergunta sobre a FAQ itaú")
    while True:
        print("Cliente:")
        question = input()
        result = qa_chain({"question": question, "chat_history": chat_history})
        chat_history.append((question, result["answer"]))
        print("FAQ_itau_bot:")
        print(result["answer"])        

Faça uma pergunta sobre a FAQ itaú
Cliente:
banana
FAQ_itau_bot:
Para desbloquear o cartão de crédito, você pode acessar o app do Itaú e digitar "desbloquear cartão" na barra de busca. Se preferir, pode ligar para a central de atendimento do banco e seguir as instruções do atendente.
Cliente:
quantos colaboradores o itau tem?
FAQ_itau_bot:
Hmm, não tenho certeza. Por favor, consulte o site do banco Itaú ou entre em contato com a central de atendimento para obter essa informação. Lembre-se de que estou aqui apenas para responder perguntas sobre o banco Itaú.
Cliente:
quero comprar um carro, como faço?
